# A1 · Cross-Encoder Reranking (Colab)

**Цель.** Обучение и применение Cross-Encoder’а для _reranking_ кандидатов: повышаем точность coarse-классификатора RuBERT на «пограничных» объявлениях и спорных парах «текст ↔ класс».  
**Ключевая идея.** В отличие от bi-encoder’а, Cross-Encoder смотрит на пару _(текст, класс/описание)_ одновременно и учится напрямую оценивать релевантность.

### Что внутри

- Подготовка парных примеров: позитивы (правильный класс) + hard-negative’ы (похожие классы).
    
- Тренировка Cross-Encoder (RuBERT-base) с BCE/Softmax-loss.
    
- Валидация качества на hold-out.
    
- Экспорт артефактов и короткая демо-ячейка инференса.
    

### Входы

- `data/train/valid/test` (или синтетические пары для CE);
    
- описания/нейминги coarse-классов.
    

### Выходы

- `data/cross_encoder_rubert/` (чекпойнт + tokenizer);
    
- (опц.) отчёт с метриками.
    

---

## Метрики (из этого ноутбука)

- **Validation:** Accuracy ≈ **0.999**, Macro-F1 ≈ **0.999**.
    
- **Test:** Accuracy ≈ **0.998**, Macro-F1 ≈ **0.998**.
    

> В ноутбуке считались _классификационные_ метрики для бинарной задачи «match/no_match». Ранжировочные (MRR/nDCG@k) не считались — если понадобятся, их можно быстро добить через scoring по всем классам и расчёт `rank_metrics(y_true, scores)`.

---

## Когда включать CE в прод

- Есть частые конфузы между близкими coarse-классами.
    
- Нужна «вторая проверка» перед возвратом top-1.
    
- Допустима цена +1 проход модели по топ-k парам.
    

**TL;DR:** CE точнее на сложных кейсах, но дороже по латентности — держим как опциональный reranker.

## 1. Проверка среды


In [1]:
import torch, platform, sys
print("Python:", sys.version.split()[0], " | Torch:", torch.__version__, " | CUDA:", torch.version.cuda)
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))


Python: 3.12.11  | Torch: 2.8.0+cu126  | CUDA: 12.6
GPU available: True
GPU name: Tesla T4


## 2. Импорты и конфиг


In [3]:
from pathlib import Path
import pandas as pd, numpy as np, json, os, re, random
from sklearn.metrics import f1_score
from datasets import Dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding, TrainingArguments, Trainer, EarlyStoppingCallback)
import evaluate, torch

SEED = 42
random.seed(SEED); np.random.seed(SEED)

BASE_MODEL = "DeepPavlov/rubert-base-cased"
MAX_LEN    = 160
BATCH_TRAIN= 16
BATCH_EVAL = 32
EPOCHS     = 3
LR         = 2e-5

ART_DIR = Path("/content/cross_encoder_rubert")
ART_DIR.mkdir(parents=True, exist_ok=True)


## 3. Данные: загрузка файлов


In [4]:

train_csv = "ce_pairs_20k_train.csv"
val_csv   = "ce_pairs_20k_val.csv"
test_csv  = "ce_pairs_20k_test.csv"
labels_json = "ce_categories.json"

from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = Path("/content/drive/MyDrive/adclassifier_ce_data")
train_csv = str(DATA_DIR / train_csv)
val_csv   = str(DATA_DIR / val_csv)
test_csv  = str(DATA_DIR / test_csv)
labels_json = str(DATA_DIR / labels_json)

df_train = pd.read_csv(train_csv)
df_val   = pd.read_csv(val_csv)
df_test  = pd.read_csv(test_csv)

print("train/val/test:", df_train.shape, df_val.shape, df_test.shape)
df_train.head(2)


Mounted at /content/drive
train/val/test: (16000, 4) (2000, 4) (2000, 4)


,text,text_b,category,label
0,Продается обувь в Краснодар материал: лен. Сос...,Категория: Обувь,Обувь,1
1,Продаю легковые автомобили — Саратов Город: Ха...,Категория: Легковые автомобили,Легковые автомобили,1


## 4. Построение датасета (HF Datasets)


In [5]:
def clean_text(s: str) -> str:
    s = re.sub(r"(\+?\d[\d\-\s]{6,}\d)", " ", str(s).lower())  # телефоны
    s = re.sub(r"\s+", " ", s).strip()
    return s

for d in (df_train, df_val, df_test):
    d["text"]   = d["text"].astype(str).map(clean_text)
    d["text_b"] = d["text_b"].astype(str).map(clean_text)
    d["label"]  = d["label"].astype(int)

ds = DatasetDict({
    "train": Dataset.from_pandas(df_train[["text","text_b","label"]], preserve_index=False),
    "validation": Dataset.from_pandas(df_val[["text","text_b","label"]], preserve_index=False),
    "test": Dataset.from_pandas(df_test[["text","text_b","label"]], preserve_index=False),
})
ds


DatasetDict({
    train: Dataset({
        features: ['text', 'text_b', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'text_b', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'text_b', 'label'],
        num_rows: 2000
    })
})

## 5. Токенизация (пары текстов)


In [6]:
tok = AutoTokenizer.from_pretrained(BASE_MODEL)

def encode(batch):
    enc = tok(batch["text"], batch["text_b"], truncation=True, max_length=MAX_LEN)
    enc["labels"] = batch["label"]
    return enc

ds_enc = ds.map(encode, batched=True, remove_columns=["text","text_b","label"])
collator = DataCollatorWithPadding(tok)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## 6. Модель и метрики


In [7]:
id2label = {0: "no_match", 1: "match"}
label2id = {"no_match": 0, "match": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, num_labels=2, id2label=id2label, label2id=label2id
)

metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, y = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=y)["accuracy"],
        "macro_f1": f1_score(y, preds, average="macro"),
    }


pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

## 7. Тренировка (Trainer + EarlyStopping)


In [8]:
from inspect import signature
def make_args(**overrides):
    base = dict(
        output_dir=str(ART_DIR / "runs"),
        learning_rate=LR,
        per_device_train_batch_size=BATCH_TRAIN,
        per_device_eval_batch_size=BATCH_EVAL,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        logging_steps=50,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        metric_for_best_model="macro_f1",
        load_best_model_at_end=True,
        report_to="none",
        fp16=torch.cuda.is_available(),
    )
    base.update(overrides)
    sig = set(signature(TrainingArguments.__init__).parameters)
    if "evaluation_strategy" not in sig and "eval_strategy" in sig:
        base["eval_strategy"] = base.pop("evaluation_strategy")
    safe = {k: v for k, v in base.items() if k in sig}
    return TrainingArguments(**safe)

args = make_args()

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["validation"],
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.0)]
)

trainer.train()
trainer.evaluate(ds_enc["test"])


/tmp/ipython-input-3513263970.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.008300,0.009003,0.998500,0.998500
2,0.000400,0.005561,0.999000,0.999000
3,0.010400,0.004698,0.999000,0.999000


{'eval_loss': 0.01253427378833294,
 'eval_accuracy': 0.998,
 'eval_macro_f1': 0.997999991999968,
 'eval_runtime': 1.6098,
 'eval_samples_per_second': 1242.396,
 'eval_steps_per_second': 39.135,
 'epoch': 3.0}

## 8. Сохранение артефактов


In [9]:
# сохраняем модель и токенайзер
trainer.save_model(str(ART_DIR))
tok.save_pretrained(str(ART_DIR))

# Сохраним вспомогательный конфиг
cfg = {
    "base_model": BASE_MODEL,
    "max_length": MAX_LEN,
    "created_by": "AdAnalyzer Cross-Encoder Colab",
    "task": "binary (text, category) match",
}
with open(ART_DIR / "ce_config.json", "w", encoding="utf-8") as f:
    json.dump(cfg, f, ensure_ascii=False, indent=2)

print("Saved to:", ART_DIR)


Saved to: /content/cross_encoder_rubert


## 9. Инференс (проверка)


In [10]:
import torch
from torch.nn.functional import softmax

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(str(ART_DIR)).to(device)
tok   = AutoTokenizer.from_pretrained(str(ART_DIR))

def score_categories(text, categories, topk=5):
    text = str(text)
    pairs = [(text, f"Категория: {c}") for c in categories]
    enc = tok([p[0] for p in pairs], [p[1] for p in pairs], return_tensors="pt", padding=True,
              truncation=True, max_length=MAX_LEN).to(device)
    with torch.no_grad():
        logits = model(**enc).logits  # [N,2]
        probs = softmax(logits, dim=-1)[:,1].detach().cpu().numpy()  # вероятность 'match'
    order = np.argsort(-probs)[:topk]
    return [(categories[i], float(probs[i])) for i in order]

# пример
try:
    with open(labels_json, "r", encoding="utf-8") as f:
        cats = json.load(f)["labels"]
except Exception:
    cats = sorted(pd.concat([df_train["category"], df_val["category"], df_test["category"]]).unique().tolist())

print(score_categories("Продам валик для покраски стен в доме", cats, topk=5)[:5])


[('Дома и дачи — аренда', 0.8360455632209778), ('Дома и дачи — продажа', 0.6117977499961853), ('Обувь', 0.0027839813847094774), ('Пылесосы', 0.002385852625593543), ('Мотоциклы и мототехника', 0.0015780552057549357)]


## 10. Экспорт (zip + download)


In [ ]:
import shutil
zip_path = "/content/cross_encoder_rubert_artifacts.zip"
shutil.make_archive(zip_path.replace(".zip",""), "zip", str(ART_DIR))
from google.colab import files
files.download(zip_path)


In [12]:
from google.colab import drive
drive.mount('/content/drive')

target_dir = "/content/drive/MyDrive/adclassifier_ce_artifacts"
os.makedirs(target_dir, exist_ok=True)

!cp /content/cross_encoder_rubert_artifacts.zip "$target_dir/"

print("файл сохранен в:", target_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
файл сохранен в: /content/drive/MyDrive/adclassifier_ce_artifacts


In [13]:

zip_path = "/content/cross_encoder_rubert_artifacts.zip"
size_mb = os.path.getsize(zip_path) / (1024*1024)
print(f"размер архива: {size_mb:.2f} MB")


размер архива: 2486.56 MB


## Итоги (A1 · Cross-Encoder)

- CE на тесте даёт **Accuracy ≈ 0.998 / Macro-F1 ≈ 0.998** на задаче match/no_match.
    
- Использовать как опциональный модуль: только для _low-margin_ случаев (топ-2 слишком близки) или офлайн для повышения релевантности автотегов.
    
- Экспортированные веса: `data/cross_encoder_rubert/`.
    

**Дальше**

- Early-exit: не вызывать CE, если `softmax_margin > τ`.
    
- Расширить hard-negatives (перестановки/near-dups).
    